In [69]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [74]:
df = pd.read_csv('./cleaned_train.csv', index_col=0)

In [75]:
df.head()

,gold_label,sentence1,sentence2,sentence1_embedd,sentence2_embedd
0,0,read slate s take jacksons findings,slate opinion jacksons findings,"[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0....","[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0...."
1,1,gays lesbians,heterosexuals,"[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0....","[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0...."
2,1,fun adults children,fun children,"[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0....","[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0...."
3,0,issues data synthesis,problems data synthesis,"[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0....","[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0...."
4,0,well see television also,see television well,"[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0....","[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0...."


In [78]:
output_labels = df['gold_label'].as_matrix()
sentence1_embedd = df['sentence1_embedd'].as_matrix()
sentence2_embedd = df['sentence2_embedd'].as_matrix()

/home/born-2-code/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/born-2-code/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/born-2-code/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
def getbatch(batch_size):
    for i in range(len(output_labels)// batch_size):
        index = i * batch_size
        index_end = index + batch_size
        yeild sentence1_embedd[index: index_end], sentence2_embedd[index: index_end], output_labels[index: index_end]

SyntaxError: invalid syntax (<ipython-input-79-82667ffdb0c2>, line 5)

In [58]:
"""
size of glove vector
"""
nn1_input = 15 * 50
nn1_hidden1 = 500
nn1_output = 100
# nn1_output = 300

nn2_input = nn1_output * 2
nn2_hidden1 = 100
# nn2_hidden1 = 50
nn2_output = 3

learning_rate = 0.001

batch_size = 128
num_steps = 1000


sentence1 = tf.placeholder(tf.float32, shape=(None, nn1_input))
sentence2 = tf.placeholder(tf.float32, shape=(None, nn1_input))

output = tf.placeholder(tf.float32, shape=(None, nn2_output))

In [59]:
nn1_weights = {
    "nn1_w1" : tf.Variable(tf.random_normal([nn1_input, nn1_hidden1])),
    "nn1_w2" : tf.Variable(tf.random_normal([nn1_hidden1, nn1_output])),
    
    "nn2_w1" : tf.Variable(tf.random_normal([nn2_input, nn2_hidden1])),
    "nn2_w2" : tf.Variable(tf.random_normal([nn2_hidden1, nn2_output]))
}

nn1_biases = {
    "nn1_b1" : tf.Variable(tf.random_normal([nn1_hidden1])),
    "nn1_b2" : tf.Variable(tf.random_normal([nn1_output])),
    
    "nn2_b1" : tf.Variable(tf.random_normal([nn2_hidden1])),
    "nn2_b2" : tf.Variable(tf.random_normal([nn2_output]))
}

In [60]:
def nn1_encode(x):
    val = tf.nn.relu(tf.matmul(x, nn1_weights["nn1_w1"]) + nn1_biases["nn1_b1"])
    return tf.nn.relu(tf.matmul(val, nn1_weights["nn1_w2"]) + nn1_biases["nn1_b2"])

e1 = nn1_encode(sentence1)
e2 = nn1_encode(sentence2)

In [61]:
nn2_input = tf.concat([e1, e2], 1)
## check diamentions

In [62]:
def nn2_encode(x):
    val = tf.nn.relu(tf.matmul(x, nn1_weights["nn2_w1"]) + nn1_biases["nn2_b1"])
    return tf.nn.relu(tf.matmul(val, nn1_weights["nn2_w2"]) + nn1_biases["nn2_b2"])

In [63]:
predictions = nn2_encode(nn2_input)

In [64]:
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predictions, labels=nn2_output))

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=predictions, labels=output))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [65]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(predictions, 1), tf.argmax(output, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [68]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_sentence1, batch_sentence2, batch_y = getbatch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={sentence1: batch_sentence1,sentence2: batch_sentence2, output: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={sentence1: batch_sentence1,sentence2: batch_sentence2, output: batch_y})
            print("Step " + str(step) + ", Step Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={sentence1: test_sentence1,sentence2: test_sentence2, output: test_y}))

TypeError: 'NoneType' object is not iterable